In [2]:
%pip --queit install llama-index

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\mix\.conda\envs\agents\python.exe -m pip <command> [options]

no such option: --queit


## 使用Ollama

In [3]:
%pip install --quiet llama-index-llms-ollama llama-index-embeddings-ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# 设置 LLM 模型（如 llama3.1）
Settings.llm = Ollama(model="qwen2.5:7b", request_timeout=360.0)

# 设置嵌入模型（如 nomic-embed-text）
Settings.embed_model = OllamaEmbedding(model_name="bge-m3")

## 快速开始
* 本地部署模型文档参考： https://docs.llamaindex.ai/en/latest/getting_started/starter_example_local/
* 更多向量数据库支持文档：https://docs.llamaindex.ai/en/latest/module_guides/storing/vector_stores/
    * 如何使用这些向量数据库：https://docs.llamaindex.ai/en/latest/community/integrations/vector_stores/ 

### Rag

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("请问文章内容是关于什么的？")
print(response)

文章内容涉及了代理人工智能（Agentic AI）的概念分类、应用以及挑战。具体包括几个实际应用场景，如智能临床决策支持系统、农业机器人协调作业、企业网络安全响应等。这些例子展示了代理人工智能在复杂动态环境中的自主任务协调能力，并强调了中央调度器管理、共享记忆模块以及反馈机制的重要性。


In [ ]:
## 上述每一次都会加载文档并构建
## 可以从数据库中加载
# Save the index

## 将上述的内容保存 
index.storage_context.persist("storage")

In [7]:
# Later, load the index
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(
    storage_context,
    # we can optionally override the embed_model here
    # it's important to use the same embed_model as the one used to build the index
    # embed_model=Settings.embed_model,
)
query_engine = index.as_query_engine(
    # we can optionally override the llm here
    # llm=Settings.llm,
)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage\index_store.json.


### Agent Example


In [8]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.ollama import Ollama

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=Ollama(
        model="Qwen2.5:7b",
        request_timeout=360.0,
        # Manually set the context window to limit memory usage
        context_window=8000,
    ),
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)


async def main():
    # Run the agent
    response = await agent.run("What is 1234 * 4567?")
    print(str(response))

await main()

The result of 1234 multiplied by 4567 is 5635678.


### 增加流式输出
不同模型的对应的流式输出支持不同，因此可以直接设置对应的参数即可。

#### 使用chat_engine或者llm自带
chat_engine自动集成了向量数据库

In [16]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.ollama import Ollama

# 1. 创建支持流式的LLM实例（关键修改）
custom_llm = Ollama(
    model="qwen2.5:7b",
    streaming=True,  # 必须启用流式标志[1,4](@ref)
    temperature=0.7,  # 控制输出多样性[1](@ref)
    request_timeout=120.0  # 避免超时中断[6](@ref)
)

# 2. 删除冗余调用（custom_llm.predict() 不必要）
# 直接使用流式接口

# 3. 正确的流式调用方式
print("测试流式输出:")
for token in custom_llm.stream_complete(prompt="你是谁？"):  # 使用stream_complete[1,2](@ref)
    print(token.delta, end="", flush=True)  # 使用delta获取增量内容[2](@ref)

# 4. 创建带记忆的聊天引擎
memory = ChatMemoryBuffer(token_limit=4000)  # 控制上下文长度[8](@ref)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    streaming=True,  # 引擎层启用流式[2](@ref)
    llm=custom_llm,
    memory=memory
)

# 5. 流式对话执行
print("\n\n对话引擎测试:")
streaming_response = chat_engine.stream_chat("什么是Agentic AI")
for token in streaming_response.response_gen:  # 正确访问响应生成器[2](@ref)
    print(token, end="", flush=True)  # 实时输出

测试流式输出:
我是Qwen，一个由阿里云开发的语言模型助手。我在这里为了提供帮助和回答问题而设计的，旨在与用户进行对话，并尽可能准确和有帮助地回应您的需求。如果您有任何问题或需要帮助，请随时告诉我！

对话引擎测试:
Agentic AI 是一种新兴的智能系统类别，它扩展了传统AI代理的能力，使其能够通过结构化的沟通、共享记忆以及动态的角色分配来实现多个智能实体之间的协作目标。与单一的AI代理模型相比，在复杂、动态、多步骤或合作场景中的应用案例日益要求上下文保留、任务依赖性和适应性。因此，Agentic AI 系统代表了一种新兴的智能化架构类型，在这种架构中，多个专业化的代理可以协同工作以实现共同目标。

具体来说，与传统的AI代理相比，Agentic AI系统具有以下特点：
1. **协作机制**：支持多个智能实体之间的协调和合作。
2. **结构化沟通**：通过有效的信息交流来促进任务执行和目标达成。
3. **共享记忆**：允许各智能体间共享相关信息以增强整体性能。
4. **动态角色分配**：根据需要调整各个代理的角色和职责。

这些特性使得Agentic AI系统能够更好地应对复杂多变的环境，并实现更加灵活、智能化的目标追求。

### 记忆

In [17]:
# # 4. 创建带记忆的聊天引擎
# memory = ChatMemoryBuffer(token_limit=4000)  # 控制上下文长度[8](@ref)
print(memory)

chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='什么是Agentic AI')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Agentic AI 是一种新兴的智能系统类别，它扩展了传统AI代理的能力，使其能够通过结构化的沟通、共享记忆以及动态的角色分配来实现多个智能实体之间的协作目标。与单一的AI代理模型相比，在复杂、动态、多步骤或合作场景中的应用案例日益要求上下文保留、任务依赖性和适应性。因此，Agentic AI 系统代表了一种新兴的智能化架构类型，在这种架构中，多个专业化的代理可以协同工作以实现共同目标。\n\n具体来说，与传统的AI代理相比，Agentic AI系统具有以下特点：\n1. **协作机制**：支持多个智能实体之间的协调和合作。\n2. **结构化沟通**：通过有效的信息交流来促进任务执行和目标达成。\n3. **共享记忆**：允许各智能体间共享相关信息以增强整体性能。\n4. **动态角色分配**：根据需要调整各个代理的角色和职责。\n\n这些特性使得Agentic AI系统能够更好地应对复杂多变的环境，并实现更加灵活、智能化的目标追求。')])]}) chat_store_key='chat_history' token_limit=4000 tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all')


## 结构化数据提取
https://docs.llamaindex.ai/en/latest/use_cases/extraction/
